# UFO

<font color='#660066'>
   
### Table of Contents

- [Data](#Data)
    - [Scraping from UFORC](#Scraping-from-UFORC)
    - [Data Format](#data_format)
    - [Cleaing Data](#cleaning_pdata)
- [Basic anlayis](#Basic-Anylsis)
- [Milestone 3](#milestones)

In [1]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from lib.scraping import fetch_shape_urls,  \
                     fetch_all_sighting_urls, \
                     store_to_json, \
                     build_report_dataframe
from lib.postprocessing import get_time_of_occurrence, \
                           to_datetime_add_year, \
                           replace_empty_with_nan, \
                           distribution_histogram 
from lib.cleaning import clean_data, \
                        clean_duration_get_min
from lib.nuforc_analysis import *

In [3]:
data_folder = "data/"

file_url_list = data_folder+'full_list_of_urls.json'

report_file = 'jurgis_full.csv'

<font color='#660066'>
    
## Loading data

of course we dont want so scrape every time

In [4]:
df_reports = pd.read_csv(data_folder+report_file)
df_reports.sample(1).head()

,Unnamed: 0,Duration,Location,Occurred,Posted,Reported,Shape,Summary,url
49257,38654,7 minutes,"Jackson, OH",8/11/2013 22:00 (Entered as : 8-11-13 22:00),8/30/2013,8/12/2013 5:13:06 PM 17:13,Fireball,"['2 round fireballs moving south', 'I went out...",http://www.nuforc.org/webreports/101/S101120.html


In [5]:
df_reports, df_madarr = clean_data(df_reports)

In [147]:
df_reports = df_reports.set_index('url')

In [148]:
df_reports.head()

,Duration,Location,State,Occurred,Posted,Reported,Shape,Summary,nuforc_note,link
url,,,,,,,,,,
http://www.nuforc.org/webreports/143/S143627.html,10 minutes,Lutz,FL,2018-10-30 23:30:00,11/9/2018,2018-10-30 20:54:00,,['Blinking white light that moved every few se...,[((NUFORC Note: We have attempted to contact ...,[]
http://www.nuforc.org/webreports/089/S89377.html,4-5 minutes,Bridgewater,NJ,2011-11-23 00:45:00,6/5/2012,2012-06-05 12:37:00,,['EXPLAINED: Bright lights in Somerset county ...,[((NUFORC Note: Witness elects to remain tota...,[http://www.nj.com/news/index.ssf/2011/11/brig...
http://www.nuforc.org/webreports/019/S19862.html,2 min,South Fork,CO,2001-10-04 14:00:00,3/21/2003,2001-10-15 13:27:00,Cigar,"['Shinny Cigar shape in sky on clear day.', 'D...",[],[]
http://www.nuforc.org/webreports/121/S121007.html,10 minutes,Hattiesburg,MS,2014-12-17 20:00:00,8/13/2015,2015-08-09 21:11:00,Triangle,['I saw a triangular UFO with red and white li...,[],[]
http://www.nuforc.org/webreports/116/S116071.html,5-10 seconds,LaBelle,FL,2014-12-17 17:19:00,12/18/2014,2014-12-17 18:21:00,Triangle,"['White Triangle in South Florida', 'My co-wor...",[],[]


In [7]:
df_dummy = df_reports.copy()

In [41]:
def clean_duration_get_min(df):
    """
    Cleans the duration section by dropping all values, which are more than a week
    Cleans the time formats and converts all of the to 'min'
    By cleaning the data we are left with 86% of the intial data,
    Which is reasonable for us, as initially there were about 10000 different
    Time formats
    Returns:
        --cleaned series with updated durations in min
    """
    df_dummy = df.Duration.copy()
    df_dummy = df_dummy.to_frame()
    df_dummy.Duration = df_dummy.Duration.str.lower()
    df_dummy = df_dummy.dropna()
    df_dummy['time_format'] = df_dummy.Duration
    hr_time = df_dummy.copy()
    hr_time.Duration = hr_time.Duration[hr_time.Duration.str.contains('1 hr')]
    hr_time = hr_time.dropna()
    hr_time.Duration = 60
    hr_time.time_format = 'min'
    hrs_time = df_dummy.copy()
    hrs_time.Duration = hrs_time.Duration[hrs_time.Duration.str.contains('hrs')]
    hrs_time = hrs_time.dropna()
    hrs_time.Duration = hrs_time.Duration.str.replace('[:-]', '.')
    hrs_time.Duration = hrs_time.Duration.str.replace(' 1/2', '.5')
    hrs_time.Duration = hrs_time.Duration.str.replace('/', '.')
    repl = lambda m: m.group('one')
    hrs_time.Duration = hrs_time.Duration.str.replace(r"(?P<one>.*)hrs.*", repl, regex=True)
    hrs_time.Duration = hrs_time.Duration.str.replace(r"[^0-9]*(?P<one>[0-9][0-9 \.]*)[\w\. ]*", repl, regex=True)
    hrs_time.Duration = hrs_time.Duration.str.replace('[+a-z ]', '')
    hrs_time.time_format = 'hour'
    manual_drop_hrs = [111938, 97125, 92361, 87440, 110228, 45333, 80653, 58194, 55810, 
                       54939, 110807, 54694, 30841, 57254]
    hrs_time = hrs_time.drop(manual_drop_hrs)
    hrs_time.Duration = hrs_time.Duration[~(hrs_time.Duration == '')]
    hrs_time = hrs_time.dropna()
    hrs_time.Duration = hrs_time.Duration.astype(float)
    hrs_time.Duration = hrs_time.Duration*60
    hrs_time.time_format = 'min'
    hour_time = df_dummy.copy()
    hour_time.Duration = hour_time.Duration[hour_time.Duration.str.contains('hour')]
    hour_time = hour_time.dropna()
    hour_time.Duration = hour_time.Duration.str.replace('[:-]', '.')
    hour_time.Duration = hour_time.Duration.str.replace(' 1/2', '.5')
    hour_time.Duration = hour_time.Duration.str.replace('/', '.')
    hour_time.Duration = hour_time.Duration.str.replace(r"(?P<one>.*)hour.*", repl, regex=True)
    hour_time.Duration = hour_time.Duration.str.replace(r"[^0-9]*(?P<one>[0-9][0-9 \.]*)[\w\. ]*", repl, regex=True)
    hour_time.Duration = hour_time.Duration.str.replace('[+a-z ?\(\)&"`,!>;]', '')
    hour_time.Duration = hour_time.Duration.str.replace('([.]{2,})', '')
    manual_drop_hours = [109182, 102324, 96844, 93487, 93353, 80915, 88291, 78633,
                         63452, 66319, 67243, 44816, 22126, 30095, 68130, 91121, 
                        80095, 71731, 61714, 38278, 24674, 107114, 111918]
    hour_time = hour_time.drop(manual_drop_hours)
    hour_time.Duration = hour_time.Duration.str.replace('\.$', '')
    hour_time.time_format = 'hour'
    hour_time.Duration = hour_time.Duration[~(hour_time.Duration == '')]
    hour_time = hour_time.dropna()
    hour_time.Duration = hour_time.Duration.astype(float)
    hour_time.Duration = hour_time.Duration*60
    hour_time.time_format = 'min'
    minutes_time = df_dummy.copy()
    minutes_time.Duration = minutes_time.Duration.dropna()[minutes_time.Duration.dropna().str.contains('min')]
    minutes_time.time_format = 'min'
    minutes_time = minutes_time.dropna()
    minutes_time.Duration = minutes_time.Duration.str.replace(' 1/2', '.5')
    minutes_time.Duration = minutes_time.Duration.str.replace('to', '-')
    minutes_time.Duration = minutes_time.Duration.str.replace('or', '-')
    minutes_time.Duration = minutes_time.Duration.str.replace('[/:]', '.')
    minutes_time['Duration'] = minutes_time.Duration.str.replace(r"(?P<one>.*)hour.*", repl, regex=True)
    minutes_time['Duration'] = minutes_time.Duration.str.replace(r"[^0-9]*(?P<one>[0-9][0-9 -\.]*)[\w\. ]*", repl, regex=True)
    minutes_time['Duration'] = minutes_time.Duration.str.replace('[+a-z ?±*\(\)½!<>\\\\=~&"á`,\'!>;\[\]@]', '')
    minutes_time['Duration'] = minutes_time.Duration.str.replace('([.]{2,})', '')
    minutes_time['Duration'] = minutes_time.Duration.str.replace('([-]{2,})', '')
    minutes_manual_drop = [2708, 7632, 17491, 26323, 17743, 19051, 21642, 25011,
                           27849, 28293, 35538, 39992, 75636, 78172, 82754, 93849,
                           95787, 4126, 24350, 95915, 102106, 4627, 85033, 85683, 
                           14141, 16443, 69234, 77029, 101165, 42895, 19145, 56129, 
                          75795, 115300, 113583, 112874, 105620, 88135, 87478, 75795, 
                          60077, 55454, 78279, 113942, 43457, 4597, 6962, 9217, 19169, 
                           32546, 41128, 61391, 63575, 82235, 101547, 32357, 89937]
    minutes_time = minutes_time.drop(minutes_manual_drop)
    minutes_time['Duration'] = minutes_time.Duration.str.replace('\-$', '')
    minutes_time.Duration = minutes_time.Duration.str.replace('\.$', '')
    minutes_time.Duration = minutes_time.Duration.str.replace('\.-$', '')
    minutes_time.Duration = minutes_time.Duration[~(minutes_time.Duration == '')]
    minutes_time = minutes_time.dropna()
    extended_duration = minutes_time[minutes_time.Duration.str.contains('-')]
    seperated_duration = extended_duration.Duration.str.split('-')
    seperated_duration = seperated_duration[~(seperated_duration.str[0] == '')]
    seperated_duration = seperated_duration[~(seperated_duration.str[1] == '')]
    calculated_averages_of_durations = (seperated_duration.str[0].astype(float)+seperated_duration.str[1].astype(float))/2
    minutes_time.update(calculated_averages_of_durations)
    minutes_time.Duration = minutes_time.Duration.astype(str).str.replace('[-]', '')
    minutes_time.Duration = minutes_time.Duration[~(minutes_time.Duration == '')]
    minutes_time = minutes_time.dropna()
    minutes_time.Duration = minutes_time.Duration.astype(float)

    sec_time = df_dummy.copy()
    sec_time.Duration = sec_time.Duration.dropna()[sec_time.Duration.dropna().str.contains('sec')]
    sec_time.time_format = 'sec'
    sec_time = sec_time.dropna()
    sec_time.Duration = sec_time.Duration.str.replace(' 1/2', '.5')
    sec_time.Duration = sec_time.Duration.str.replace('to', '-')
    sec_time.Duration = sec_time.Duration.str.replace('or', '-')
    sec_time.Duration = sec_time.Duration.str.replace('[/:]', '.')
    repl = lambda m: m.group('one')
    sec_time['Duration'] = sec_time.Duration.str.replace(r"(?P<one>.*)sec.*", repl, regex=True)
    sec_time['Duration'] = sec_time.Duration.str.replace(r"[^0-9]*(?P<one>[0-9][0-9 -\.]*)[\w\. ]*", repl, regex=True)
    sec_time['Duration'] = sec_time.Duration.str.replace('[+a-z ?±*\(\)½!<>\\\\=~&"á`,\'!>;\[\]@]', '')
    sec_time['Duration'] = sec_time.Duration.str.replace('([.]{2,})', '')
    sec_time['Duration'] = sec_time.Duration.str.replace('([-]{2,})', '')
    sec_time['Duration'] = sec_time.Duration.str.replace('\-$', '')
    sec_manual_drop = [17067, 42074, 21247]
    sec_time = sec_time.drop(sec_manual_drop)
    sec_time.Duration = sec_time.Duration.str.replace('\.$', '')
    sec_time.Duration = sec_time.Duration.str.replace('\.-$', '')
    sec_time.Duration = sec_time.Duration[~(sec_time.Duration == '')]
    sec_time = sec_time.dropna()

    sec_extended_duration = sec_time[sec_time.Duration.str.contains('-')]
    sec_seperated_duration = sec_extended_duration.Duration.str.split('-')
    sec_seperated_duration = sec_seperated_duration[~(sec_seperated_duration.str[0] == '')]
    sec_seperated_duration = sec_seperated_duration[~(sec_seperated_duration.str[1] == '')]
    sec_calculated_averages_of_durations = (sec_seperated_duration.str[0].astype(float) + 
                                        sec_seperated_duration.str[1].astype(float))/2

    sec_time.update(sec_calculated_averages_of_durations)
    sec_time.Duration = sec_time.Duration.astype(str).str.replace('[-]', '')
    sec_time.Duration = sec_time.Duration[~(sec_time.Duration == '')]
    sec_time = sec_time.dropna()
    sec_time.Duration = sec_time.Duration.astype(float)
    sec_time.Duration = sec_time.Duration/60
    sec_time.time_format = 'min'

    duration_list = [sec_time, minutes_time, hr_time, hrs_time, hour_time]
    df_new = df.copy()
    df_new.Duration = np.nan
    for i in duration_list:
        df_new.Duration.update(i.Duration)
    
    return df_new


In [42]:
df_d = clean_duration_get_min(df_reports)

In [43]:
df_d

,Duration,Location,State,Occurred,Posted,Reported,Shape,Summary,url,nuforc_note,link
0,10.000000,Lutz,FL,2018-10-30 23:30:00,11/9/2018,2018-10-30 20:54:00,,['Blinking white light that moved every few se...,http://www.nuforc.org/webreports/143/S143627.html,[((NUFORC Note: We have attempted to contact ...,[]
4,4.500000,Bridgewater,NJ,2011-11-23 00:45:00,6/5/2012,2012-06-05 12:37:00,,['EXPLAINED: Bright lights in Somerset county ...,http://www.nuforc.org/webreports/089/S89377.html,[((NUFORC Note: Witness elects to remain tota...,[http://www.nj.com/news/index.ssf/2011/11/brig...
5,2.000000,South Fork,CO,2001-10-04 14:00:00,3/21/2003,2001-10-15 13:27:00,Cigar,"['Shinny Cigar shape in sky on clear day.', 'D...",http://www.nuforc.org/webreports/019/S19862.html,[],[]
6,10.000000,Hattiesburg,MS,2014-12-17 20:00:00,8/13/2015,2015-08-09 21:11:00,Triangle,['I saw a triangular UFO with red and white li...,http://www.nuforc.org/webreports/121/S121007.html,[],[]
7,0.125000,LaBelle,FL,2014-12-17 17:19:00,12/18/2014,2014-12-17 18:21:00,Triangle,"['White Triangle in South Florida', 'My co-wor...",http://www.nuforc.org/webreports/116/S116071.html,[],[]
8,0.500000,Maryland Heights,MO,2014-12-16 17:15:00,12/18/2014,2014-12-16 21:33:00,Triangle,['I was driving on my motorcycle on hwy 141 at...,http://www.nuforc.org/webreports/116/S116064.html,[],[]
9,NaN,Fort Mill,SC,2014-12-16 17:00:00,12/18/2014,2014-12-16 19:01:00,Triangle,['Triangular craft with lights are corners wit...,http://www.nuforc.org/webreports/116/S116060.html,[((NUFORC Note: Witness elects to remain tota...,[]
10,5.000000,Lenoir City,TN,2014-12-15 05:30:00,1/7/2015,2014-12-28 18:11:00,Triangle,['My wife and I were on our way to work. She s...,http://www.nuforc.org/webreports/116/S116228.html,[],[]
11,30.000000,Augusta,GA,2014-12-14 22:00:00,12/18/2014,2014-12-14 19:16:00,Triangle,"[""I was coming home from work when I noticed 3...",http://www.nuforc.org/webreports/115/S115995.html,[],[]
12,10.000000,Meriden,CT,2014-12-14 09:50:00,12/18/2014,2014-12-14 19:30:00,Triangle,['1 Very Large triangle shaped 5 lights around...,http://www.nuforc.org/webreports/115/S115994.html,[],[]
